Convert Keras .h5 weights to TensorFlow.js file<br>
[Importing a Keras model into TensorFlow.js](https://www.tensorflow.org/js/tutorials/conversion/import_keras)

In [1]:
! pip install -q tensorflowjs

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 144.4MB 62kB/s 
     |████████████████████████████████| 92kB 7.8MB/s 
     |████████████████████████████████| 256kB 45.0MB/s 
     |████████████████████████████████| 921kB 41.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [2]:
import numpy as np
from tensorflow.keras.applications import MobileNet
import tensorflowjs as tfjs

### Fetch .h5 file from URL or google disk

In [ ]:
# Mount your google disk if you need it
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Download model and rename it. 
# Modify the path as you need it.
! git clone --single-branch --branch keras_model https://github.com/Gooogr/Doodle-Recognition-Web.git
! cp /content/Doodle-Recognition-Web/model/kaggle_model.h5 \
     /content/weights.h5 --force #don't change this part

Cloning into 'Doodle-Recognition-Web'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 118 (delta 38), reused 100 (delta 27), pack-reused 0
Receiving objects: 100% (118/118), 12.80 MiB | 31.66 MiB/s, done.
Resolving deltas: 100% (38/38), done.


### Create model and load weights

In [4]:
SIZE = 64
NCATS = 340 #Hardcoded parameter from dataset

In [5]:
model = MobileNet(input_shape=(SIZE, SIZE, 1), alpha=1., weights='/content/weights.h5', classes=NCATS)
model.summary()

Model: "mobilenet_1.00_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 65, 65, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 32, 32, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 32, 32, 32)  

In [6]:
# Sanity check
noise = np.random.randn(1, SIZE, SIZE, 1)
preds = model.predict(noise)
assert preds.shape != [1, 340]
print('Sanity check passed!')

Sanity check passed!


### Convert model to TensorFlow.js

In [7]:
# convert and zip files
! mkdir /content/weights_js
tfjs.converters.save_keras_model(model, '/content/weights_js')
! zip -r converted_weights.zip /content/weights_js

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


  adding: content/weights_js/ (stored 0%)
  adding: content/weights_js/group1-shard3of4.bin (deflated 7%)
  adding: content/weights_js/group1-shard4of4.bin (deflated 7%)
  adding: content/weights_js/model.json (deflated 95%)
  adding: content/weights_js/group1-shard2of4.bin (deflated 7%)
  adding: content/weights_js/group1-shard1of4.bin (deflated 7%)
